In [7]:
#import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy

import pymongo
from pymongo import MongoClient

In [8]:
# Initialize mongoDB client
client = MongoClient()
db = client.yelp

In [98]:
# Gather businesses data from DB
# create new collection with split categories
db.temp_businesses.drop()
agr = [{'$project': {'business_id':1,'name':1,'address':1,'city':1,'state':1,'postal_code':1,'latitude':1,'longitude':1,'stars':1,'review_count':1,'is_open':1,'attributes':1,'categories': {'$split':["$categories",","]},'hours':1}},{'$out': 'temp_businesses'}]
businesses_agr = db.businesses.aggregate(agr)
businesses_collection = db.temp_businesses
businesses_df = pd.DataFrame(list(businesses_collection.find()))
businesses_count = businesses_df.shape[0]

In [99]:
# Obtain random business from businesses dataframe
random_number = np.random.randint(1,businesses_count)
business = businesses_df.iloc[random_number]
business

_id                                      5e925b5725c46ba0c36a9862
business_id                                1zJLGL-GVcxnkp6dZWFCTQ
name                                                      McBride
address                                          22246 N 19th Ave
city                                                      Phoenix
state                                                          AZ
postal_code                                                 85027
latitude                                                   33.689
longitude                                                  -112.1
stars                                                         4.5
review_count                                                    6
is_open                                                         1
attributes      {'BusinessAcceptsCreditCards': 'True', 'ByAppo...
hours           {'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...
categories      [Damage Restoration,  Roofing,  Home Services,...
Name: 8629

In [100]:
# Get categories array
categories = business.get('categories')

['Damage Restoration', ' Roofing', ' Home Services', ' Contractors']

In [101]:
# Get Businesses in categories ($all for all in categories, $in for any in categories)
businesses_in_categories_df = pd.DataFrame(list(businesses_collection.find({ "categories": {"$all": categories} })))
businesses_in_categories_df

,_id,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours,categories
0,5e925b5625c46ba0c369ef13,8Sh7dIXzYeXe3MoY15RCPw,Nevada Construction,,Henderson,NV,89052,35.952556,-115.022874,3.0,4,0,"{'BusinessAcceptsCreditCards': 'False', 'Wheel...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:0', '...","[Damage Restoration, Shopping, Home & Garden..."
1,5e925b5625c46ba0c369f348,z8pA5MJIpqZFDN0H3eFdrA,ASAP Restoration LLC,"20701 N Scottsdale Rd, Ste 107-292",Scottsdale,AZ,85255,33.674682,-111.923515,4.5,51,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...","[Damage Restoration, Contractors, Home Inspe..."
2,5e925b5625c46ba0c36a1003,P_uEm_8prOXNR7O5jYFdUg,MRM Roofing,"55 W Hoover Ave, Ste 8",Mesa,AZ,85210,33.388595,-111.833802,3.0,6,0,None,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","[Damage Restoration, Roofing, Home Services,..."
3,5e925b5725c46ba0c36a9862,1zJLGL-GVcxnkp6dZWFCTQ,McBride,22246 N 19th Ave,Phoenix,AZ,85027,33.689023,-112.100401,4.5,6,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...","[Damage Restoration, Roofing, Home Services,..."
4,5e925b5925c46ba0c36b2fbd,5w7whjK5mDJsbp9so4rVZQ,Arizona Construction & Restoration,2423 S 17th St,Phoenix,AZ,85034,33.424737,-112.045169,4.5,31,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","{'Monday': '6:0-5:30', 'Tuesday': '6:0-5:30', ...","[Damage Restoration, Home Services, Roofing,..."
5,5e925b5925c46ba0c36b30e9,9fFzSUHB_LM9MBGQnSGYZA,Silver City Restoration,2255 W Desert Cv,Phoenix,AZ,85029,33.585192,-112.106400,4.0,22,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...","[Damage Restoration, Contractors, Plumbing, ..."
6,5e925b5a25c46ba0c36beb80,DIY8OZ4VZ-1Xy-G_YI1zkA,Envy Restoration,116 E Broadway Rd,Mesa,AZ,85210,33.407980,-111.827870,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...","[Damage Restoration, Home Services, Contract..."
